# Download TrendMiner Context-Item Attachments

The script is designed to automate the retrieval of documents or files attached to specific context items within the TrendMiner platform.  
Context items in TrendMiner are annotations or events linked to process data, often enriched with additional information like attachments.  
By executing this script, you can programmatically download attachments associated with context items without manually navigating through the TrendMiner interface.  

> **Tip:**  
> To use this script you will need the **short key** of the context item (for example `386-3` in the screenshot), which appears as the title when you open a context item. You can also inspect the **Attachments** tab to see available attachment names.
![Context Item Example](https://userguide.trendminer.com/image/uuid-ba68e2ee-97c6-7b28-95b0-a9138c3d6623.png)

In [ ]:
import os
import requests
from typing import List, Optional, Dict

def get_context_attachments(
    key: str,
    attachment_names: Optional[List[str]] = None
) -> Dict[str, bytes]:
    """
    Fetch one or more attachments from a TrendMiner context item.

    Parameters
    ----------
    key : str
        The short key of the context item (e.g. "386-3").
    attachment_names : list of str, optional
        If provided, only attachments whose names are in this list will be downloaded.
        If None, all attachments will be downloaded.

    Returns
    -------
    dict of str to bytes
        A mapping from attachment name to its raw content bytes.

    Raises
    ------
    ValueError
        If the required environment variables KERNEL_SERVER_URL or KERNEL_USER_TOKEN are missing.
    FileNotFoundError
        If `attachment_names` is provided but one or more names are not found.
    HTTPError
        If any HTTP request returns an error status.
    """
    # read configuration from environment
    server = os.getenv("KERNEL_SERVER_URL", "").rstrip('/')
    token = os.getenv("KERNEL_USER_TOKEN", "")
    if not server or not token:
        raise ValueError(
            "Environment variables KERNEL_SERVER_URL and KERNEL_USER_TOKEN must be set"
        )

    # prepare session
    sess = requests.Session()
    sess.verify = False
    sess.headers.update({"Authorization": f"Bearer {token}"})

    # 1) fetch context-item metadata
    resp = sess.get(f"{server}/context/item/{key}")
    resp.raise_for_status()
    item = resp.json()
    context_id = item["identifier"]

    # 2) find attachments URL
    attachments_url = next(
        link["href"] for link in item.get("links", [])
        if link.get("rel") == "attachments" or "attachments" in link.get("href", "")
    )

    # 3) list attachments
    resp = sess.get(attachments_url)
    resp.raise_for_status()
    entries = resp.json().get("content", [])

    # 4) download requested attachments
    results: Dict[str, bytes] = {}
    for att in entries:
        name = att["name"]
        if attachment_names is not None and name not in attachment_names:
            continue

        att_id = att["identifier"]
        dl = sess.get(f"{server}/context/data/{context_id}/attachments/{att_id}/download")
        dl.raise_for_status()
        results[name] = dl.content

    # 5) verify all requested names were found
    if attachment_names is not None:
        missing = set(attachment_names) - set(results.keys())
        if missing:
            raise FileNotFoundError(
                f"Could not find attachments: {', '.join(sorted(missing))}"
            )

    return results

In [ ]:
# Fetch all attachments
all_atts = get_context_attachments("386-3")
print(all_atts)

# Fetch just one by name
one_att = get_context_attachments("386-3", attachment_names=["import_name"])
print(one_att["import_name"])